# Advanced Data Analytics

## Part 1 - Data Manipulation


Um dieses Notebook auszuführen wähle die jeweilige "Line of Code" und drücke **CTRL+ENTER**. Mit **SHIFT+ENTER** führst du eine Zeile aus und springst automatisch zur nächsten 

In diesem Notebook beschäftigen wir uns mit dem Paket tidyverse bzw. im speziellen mit dem Paket dplyr. Dplyr zeichnet sich durch einfache Syntax sowie dem Verknüpfen von Operationen mit dem "Pipe" Operator aus. Ziel ist es, Data Manipulation zu vereinfachen.

Eingesetzt wird dplyr und Data Manipulation vor allem zur Aufbereitung bereits vorhandener Daten. Es geht dabei darum, die Daten für weitere Analyse, Visualisierungen und auch Modelle vorzubereiten.

### Installing Packages

Ein großer Grund, warum sich R großer Beliebtheit erfreut ist die große Menge und Diversität an verfügbaren Paketen. Viele Pakete sind via CRAN (https://cran.r-project.org/) verfügbar und können intuitiv in R mit der Funktion install.packages() installiert werden. Alternativ bietet RStudio auch eine graphische Maske für das Installieren und Laden von Paketen

Die Pakete/Libraries werden dann lokal (auf dem PC des Users) gespeichert. Um die Pakete und ihre Funktionen auch nutzen zu können müssen die Libraries vor jeder Session neu importiert.

Die Pakete "tidyverse" und "ggplot2" wurden bereits vorab installiert auf den Binder Maschinen installiert.

Weitere Nützliche Pakete: 
https://support.posit.co/hc/en-us/articles/201057987-Quick-list-of-useful-R-packages

Quelle Tidyverse: https://joss.theoj.org/papers/10.21105/joss.01686


In [ ]:
#install.packages("tidyverse") #already installed

install.packages(c("DBI")) #Beispiel Paket zur Verküpfung mit SQL Datenbanken 

In [ ]:
#?DBI::dbConnect

In [ ]:
library(DBI)
#?dbConnect

In [ ]:
library(dplyr)

### Die "Pipe" %>%

Das dplyr Paket ist auf die Arbeit mit DataFrames (bzw. tibble) ausgerichtet. Alle Funktionen nehmen als ersten Input ein Dataframe.

Sobald mehr als ein Schritt ausgeführt werden soll wird der Code unübersichtlich oder es müssen mehrere Zwischentabellen verwendet werden. Mit dem Piping Operator können Funktionen sequenziell ausgeführt werden und die Daten werden "übergeben"

Syntax:
my_df %>% summary()

In [ ]:
data(mtcars)

In [ ]:
#summary(mtcars)
mtcars %>% summary()

In [ ]:
mtcars %>% head()
head(mtcars)

### Single Table Verbs


Das Ziel von dplyr ist es, einfache und sprechende Funktionen für Data Manipulation zur Verfügung zu stellen. 

Row-based functions:
* slice
* filter
* arrange

Column-based functions:
* select
* rename
* relocate
* mutate

#### Column-based-functions

* select für Selektierung von Spalten
* rename für Umbenennung von Spalten
* mutate zur Veränderung von Spalten und Erstellung von neuen Spalten
* relocate um die Reihenfolge von Spalten zu Verändern

In [ ]:
#select
head(mtcars[,c("hp","wt")])
head(mtcars %>% select(c("hp","wt")))

In [ ]:
#rename
df <- mtcars
df[,"PS"] <- df[,"hp"] #neue Spalte erstellen
df[,"hp"] <- NULL #Alte Spalte löschen
head(df)

In [ ]:
df <- mtcars
#names(mtcars) <- c('mpg', 'cyl', 'disp', 'PS', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb')
names(df)[4] <- "PS"
head(df)

In [ ]:
mtcars %>% rename("PS" = hp)

In [ ]:
#mutate
df <- mtcars
df[,"Gewicht"] <- df[,"wt"]* 1000 * 0.4535923
df[,"l/100km"] <- 235/df[,"mpg"]
head(df)

In [ ]:
df <- mtcars %>% mutate(Auto=rownames(mtcars),
    Gewicht = round(wt *1000 * 0.4536,2),             
    "l/100km" =  235/mtcars[,"mpg"])
head(df)

In [ ]:
#relocate
head(mtcars[,c(4,1,2,3,5,6,7,8,9,10,11)])
head(mtcars %>% relocate(hp))

#### Row-based-functions

* slice für Indexierung
* arrange für Sortierung
* filter für Filtering von Reihen

In [ ]:
mtcars[10:15,]
mtcars %>% slice(10:15)

In [ ]:
#arrange
sorted1 <- mtcars[order(mtcars$hp),]
sorted2 <- mtcars %>% arrange(hp)

In [ ]:
head(sorted1)
head(sorted2)

In [ ]:
sorted3 <- mtcars[order(mtcars$cyl,-mtcars$hp),]
sorted4 <- mtcars %>% arrange(cyl,-hp)

In [ ]:
head(sorted3)
head(sorted4)

In [ ]:
# filter
mtcars[mtcars$hp>200,]
mtcars %>% filter(hp>200)

In [ ]:
mtcars %>% filter(hp>200 & wt<3.5)

### Kombination von mehreren Operatoren

* Filtern von Autos mit mit Namen "Ford"
* Sortieren nach hp absteigend und nur Top 5 Ergebnisse
* Berechnung von Auto und Gewicht, Filtern auf Autos < 1000 kg und relocate von Auto an Stelle 1
* Berechnung von Auto, Gewicht, l/100km und Automatik, Übersetzung von cyl, gear, qsec und Select nur neue Spalten

In [ ]:
library(data.table)
mtcars %>% mutate(Auto=rownames(mtcars)) %>% filter(Auto %like% "Ford")

In [ ]:
mtcars %>% arrange(hp) %>% slice(1:5)

In [ ]:
mtcars %>% mutate(Auto=rownames(mtcars),
    Gewicht = round(wt *1000 * 0.4536,2)) %>%
    filter(Gewicht < 1000) %>%
    relocate(Auto)

In [ ]:
autos <- mtcars %>% 
    mutate(Auto=rownames(mtcars),
            Gewicht = round(wt *1000 * 0.4536,2),
            "l/100km" =  235/mtcars[,"mpg"],
            Typ = ifelse(am==0,"Automatik","Manuell")
            ) %>%
    rename("Zylinder" = cyl,
           "Gang" = gear,
           "Viertelmeile" = qsec) %>%
    select("Auto":"Viertelmeile")

### Summarize and Grouping

* summarize für Generalisierungen (fasst Informationen in einer Zeile zusammen)
* group_by für Gruppierungen

Achtung: group_by hat nur eine Wirkung in Kombination mit summarize

* Minimum für HP, Durchschnitt für HP und QSEC über alle Records
* Durchschnitt von MPG und QSEC sowie count über Zylinder
* Filter für Autos, die schneller als der Durchschnitt der Autos mit selber Zylinderanzahl sind.

In [ ]:
mtcars %>% summarize(hp_avg=mean(hp),
                     hp_min=min(hp),
                     qsec_avg=mean(qsec))

In [ ]:
mtcars %>% 
    group_by(cyl) %>% 
    summarize(mpg_avg = mean(mpg),
              qsec_avg = mean(qsec),
              n=n())

In [ ]:
autos %>% 
    left_join(autos %>% 
                  group_by(Zylinder) %>%
                  summarize(vm_avg = mean(Viertelmeile)),
             by="Zylinder") %>%
    filter(Viertelmeile < vm_avg) %>%
    arrange(Zylinder)